In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium 

In [2]:
CLIENT_ID = '5ST00KPV0WVABK5VSWZDIQYYMYRHVLZPJVXHAOIFZF0KUVQ3' 
CLIENT_SECRET = 'EMESEPJRYLHYHLZXAM5UFXE01J4KFRBW1ILCV43HYQFO1LWP' 
VERSION = '20180605'
LIMIT = 30

In [3]:
from geopy.geocoders import Nominatim

In [8]:
address = ' Tokyo ,JP'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

35.6828387 139.7594549


In [9]:
search_query = 'Chinese'
radius = 500
print(search_query + ' .... OK Subhan!')

Chinese .... OK Subhan!


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5ST00KPV0WVABK5VSWZDIQYYMYRHVLZPJVXHAOIFZF0KUVQ3&client_secret=EMESEPJRYLHYHLZXAM5UFXE01J4KFRBW1ILCV43HYQFO1LWP&ll=35.6828387,139.7594549&v=20180605&query=Chinese&radius=500&limit=30'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebd4dcdc8cff2001c4b0e6e'},
 'response': {'venues': [{'id': '4b7fa061f964a5204d3730e3',
    'name': 'Yakiniku Toraji (焼肉トラジ)',
    'location': {'address': '丸の内2-4-1',
     'crossStreet': '丸ビル 6F',
     'lat': 35.68138219456088,
     'lng': 139.76351737976074,
     'labeledLatLngs': [{'label': 'display',
       'lat': 35.68138219456088,
       'lng': 139.76351737976074}],
     'distance': 401,
     'postalCode': '100-6323',
     'cc': 'JP',
     'city': '千代田区',
     'state': '東京都',
     'country': '日本',
     'formattedAddress': ['丸の内2-4-1 (丸ビル 6F)', '千代田区, 東京都', '100-6323', '日本']},
    'categories': [{'id': '4bf58dd8d48988d1df931735',
      'name': 'BBQ Joint',
      'pluralName': 'BBQ Joints',
      'shortName': 'BBQ',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1589464554',
    'hasPerk': False},
   {'id': '4bbe927b98f495218388d163',
    'nam

In [13]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

C:\asda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4b7fa061f964a5204d3730e3,Yakiniku Toraji (焼肉トラジ),"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",v-1589464554,False,丸の内2-4-1,丸ビル 6F,35.681382,139.763517,"[{'label': 'display', 'lat': 35.68138219456088...",401,100-6323,JP,千代田区,東京都,日本,"[丸の内2-4-1 (丸ビル 6F), 千代田区, 東京都, 100-6323, 日本]"
1,4bbe927b98f495218388d163,Huton Chinese Deli (胡同家常菜),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1589464554,False,丸の内2-4-1,丸ビル B1F,35.680983,139.763321,"[{'label': 'display', 'lat': 35.68098335697668...",406,100-6390,JP,千代田区,東京都,日本,"[丸の内2-4-1 (丸ビル B1F), 千代田区, 東京都, 100-6390, 日本]"


In [14]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Yakiniku Toraji (焼肉トラジ),BBQ Joint,丸の内2-4-1,丸ビル 6F,35.681382,139.763517,"[{'label': 'display', 'lat': 35.68138219456088...",401,100-6323,JP,千代田区,東京都,日本,"[丸の内2-4-1 (丸ビル 6F), 千代田区, 東京都, 100-6323, 日本]",4b7fa061f964a5204d3730e3
1,Huton Chinese Deli (胡同家常菜),Chinese Restaurant,丸の内2-4-1,丸ビル B1F,35.680983,139.763321,"[{'label': 'display', 'lat': 35.68098335697668...",406,100-6390,JP,千代田区,東京都,日本,"[丸の内2-4-1 (丸ビル B1F), 千代田区, 東京都, 100-6390, 日本]",4bbe927b98f495218388d163


In [15]:
dataframe_filtered.name

0       Yakiniku Toraji (焼肉トラジ)
1    Huton Chinese Deli (胡同家常菜)
Name: name, dtype: object

In [16]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Subhan Suggested Location for venue',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map

In [17]:
address = ' Toronto, CA'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [18]:
search_query = 'Chinese'
radius = 500
print(search_query + ' .... OK Subhan!')

Chinese .... OK Subhan!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5ST00KPV0WVABK5VSWZDIQYYMYRHVLZPJVXHAOIFZF0KUVQ3&client_secret=EMESEPJRYLHYHLZXAM5UFXE01J4KFRBW1ILCV43HYQFO1LWP&ll=43.6534817,-79.3839347&v=20180605&query=Chinese&radius=500&limit=30'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebd4eb7be61c9001b66de19'},
 'response': {'venues': [{'id': '4f04779a02d5cce0cfc06151',
    'name': 'Chinese Visa Application Service Center',
    'location': {'address': '393 University Ave, Suite 1501',
     'crossStreet': 'in University Centre',
     'lat': 43.65402839343005,
     'lng': -79.38736458967021,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65402839343005,
       'lng': -79.38736458967021}],
     'distance': 282,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['393 University Ave, Suite 1501 (in University Centre)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d126941735',
      'name': 'Government Building',
      'pluralName': 'Government Buildings',
      'shortName': 'Government',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
       'suffix': '.png'},
      'primary': True}],


In [21]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

C:\asda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,venuePage.id,location.neighborhood
0,4f04779a02d5cce0cfc06151,Chinese Visa Application Service Center,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1589464754,False,"393 University Ave, Suite 1501",in University Centre,43.654028,-79.387365,"[{'label': 'display', 'lat': 43.65402839343005...",282,CA,Toronto,ON,Canada,"[393 University Ave, Suite 1501 (in University...",NaN,NaN,NaN
1,4b2027b5f964a520f82d24e3,Hong Shing Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1589464754,False,195 Dundas St W,at University Ave,43.654925,-79.387089,"[{'label': 'display', 'lat': 43.65492521335936...",300,CA,Toronto,ON,Canada,"[195 Dundas St W (at University Ave), Toronto ...",M5G 1C7,60327598,NaN
2,5202d246498e9649c88150b7,Toronto Chinese Academy,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",v-1589464754,False,133 Richmond Street West,York Street,43.650140,-79.384857,"[{'label': 'display', 'lat': 43.65014006138457...",379,CA,Toronto,ON,Canada,"[133 Richmond Street West (York Street), Toron...",M5H 2L3,NaN,Financial District
3,52a7ae41498eed3af4d0a3fa,Yueh Tung Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1589464754,False,126 Elizabeth St.,Dundas St.,43.655281,-79.385337,"[{'label': 'display', 'lat': 43.65528126342919...",229,CA,Toronto,ON,Canada,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...",NaN,NaN,NaN
4,4c15280ea5eb76b0c4d9c2b7,Tasty Chinese Food,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1589464754,False,Village by the Grange,NaN,43.653757,-79.390757,"[{'label': 'display', 'lat': 43.65375663396815...",550,CA,Toronto,ON,Canada,"[Village by the Grange, Toronto ON, Canada]",NaN,NaN,NaN


In [22]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",in University Centre,43.654028,-79.387365,"[{'label': 'display', 'lat': 43.65402839343005...",282,CA,Toronto,ON,Canada,"[393 University Ave, Suite 1501 (in University...",NaN,NaN,4f04779a02d5cce0cfc06151
1,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,at University Ave,43.654925,-79.387089,"[{'label': 'display', 'lat': 43.65492521335936...",300,CA,Toronto,ON,Canada,"[195 Dundas St W (at University Ave), Toronto ...",M5G 1C7,NaN,4b2027b5f964a520f82d24e3
2,Toronto Chinese Academy,University,133 Richmond Street West,York Street,43.650140,-79.384857,"[{'label': 'display', 'lat': 43.65014006138457...",379,CA,Toronto,ON,Canada,"[133 Richmond Street West (York Street), Toron...",M5H 2L3,Financial District,5202d246498e9649c88150b7
3,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,Dundas St.,43.655281,-79.385337,"[{'label': 'display', 'lat': 43.65528126342919...",229,CA,Toronto,ON,Canada,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...",NaN,NaN,52a7ae41498eed3af4d0a3fa
4,Tasty Chinese Food,Chinese Restaurant,Village by the Grange,NaN,43.653757,-79.390757,"[{'label': 'display', 'lat': 43.65375663396815...",550,CA,Toronto,ON,Canada,"[Village by the Grange, Toronto ON, Canada]",NaN,NaN,4c15280ea5eb76b0c4d9c2b7


In [23]:
dataframe_filtered.name

0    Chinese Visa Application Service Center
1              Hong Shing Chinese Restaurant
2                    Toronto Chinese Academy
3               Yueh Tung Chinese Restaurant
4                         Tasty Chinese Food
Name: name, dtype: object

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Subhan Suggested Location for venue',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map